if not yet available some libraries have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- python3 -m pip install vtk --user

In [1]:
import numpy as np
from scipy import constants
from MeshedFields import *

In [2]:
source = MeshedField.CircularMesh(R=0.075, ratio=1.0, lcar=0.1)
print("%d points" % len(source.points))
print("%d triangles" % len(source.triangles))

459 points
848 triangles


In [3]:
source.ShowMeshedField()

The shape of a off-axis parabola follows a curve $z = \frac{r^2}{2 r_{fok}}$. $r_{fok}$ is the focal length of the parabola which is the distance of the (parallel) incidence axis of the beam and the rotational symmetry axis of the parabola (the z-axis). The distance from the parabola vertex to the focus is $r_{fok}/2$.<br>
We compute the parabolic shape in a local coordinate system centered at the intersection of the incidence axis with the shape. In this local coordinate system, the focus is placed at $[r_{fok},0,0]$ and the vertex of the parabola at $[r_{fok},0,r_{fok}/2]$. A parallel beam incident from $[0,0,-z]$ (propagating in z direction) is collected into the focus at $[r_{fok},0,0]$.

In [9]:
def parabola(p,rfok):
    """
    Move a point p along the z axis so that is lies on the surface
    of an off-axis parabola with focal length rfok.
    """
    x = rfok-p[0]
    y = p[1]
    r = np.sqrt(x*x+y*y)
    z = -(r*r/(2.0*rfok)-0.5*rfok)
    return np.array([p[0],p[1],z])

pts = [parabola(p,0.1) for p in source.points]
para = MeshedField(pts, source.triangles)

In [10]:
para.ShowMeshedField(showCenters=False,showAxes=True)

In [11]:
filename="Parabola.h5"
para.WriteMeshedField(filename)